In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.arima.model import ARIMA
from pmdarima import auto_arima
from sklearn.metrics import mean_absolute_error, mean_squared_error
import yfinance as yf
from datetime import timedelta

# Set plotting style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = [12, 6]

# Option 2: Read from CSV
BTC = pd.read_csv("C://TS AUTOMATION//BTC.csv")

# Convert date column to datetime
BTC['Datetime'] = pd.to_datetime(BTC['Datetime'])

# Initial data plot - equivalent to plot_time_series in R
plt.figure(figsize=(14, 7))
plt.plot(BTC['Datetime'], BTC['Close'])
plt.title('Bitcoin Price History')
plt.xlabel('Date')
plt.ylabel('Price (USD)')
plt.tight_layout()
plt.show()


In [ ]:
import warnings
warnings.filterwarnings("ignore")

# Select date and price columns
BTC1 = BTC[['Datetime', 'Close']].copy()
BTC1.set_index('Datetime', inplace=True)

# Split data into training and testing sets
split_point = len(BTC1) - int(len(BTC1) * 0.2)  # 80% training, 20% testing
train = BTC1.iloc[:split_point]
test = BTC1.iloc[split_point:]

print(f"Training set: {train.shape[0]} records")
print(f"Testing set: {test.shape[0]} records")

# Model development - auto_arima equivalent to R's auto_arima
model = auto_arima(train, 
                   seasonal=False,  
                   stepwise=True,
                   suppress_warnings=True,
                   d=None, 
                   max_p=5,
                   max_q=5)

print(model.summary())

# Fit the model
model_fit = model.fit(train)

# Testing forecast - forecast the test period
test_forecast = model_fit.predict(n_periods=len(test))
test_forecast = pd.Series(test_forecast, index=test.index)

# Create confidence intervals (95%)
# Assuming standard error is roughly 1.96 times the model's in-sample standard error
forecast_std = np.sqrt(model_fit.aicc()) / 10  # Simplified approximation
test_forecast_upper = test_forecast + 1.96 * forecast_std
test_forecast_lower = test_forecast - 1.96 * forecast_std

# Calculate accuracy metrics
mae = mean_absolute_error(test, test_forecast)
rmse = np.sqrt(mean_squared_error(test, test_forecast))
mape = np.mean(np.abs((test['Close'] - test_forecast) / test['Close'])) * 100

print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")

# Plot test forecast
plt.figure(figsize=(14, 7))
plt.plot(train.index, train['Close'], label='Training Data')
plt.plot(test.index, test['Close'], label='Actual Test Data')
plt.plot(test.index, test_forecast, label='Forecast', color='red')
plt.fill_between(test.index, test_forecast_lower, test_forecast_upper, color='red', alpha=0.1, label='95% Confidence Interval')
plt.title('Bitcoin Price Forecast - Test Period')
plt.xlabel('Date')
plt.ylabel('Price (USD)')
plt.legend()
plt.tight_layout()
plt.show()

# Refit model on the entire dataset
full_model = model.fit(BTC1)

# Generate future forecast - 20 periods ahead
future_periods = 20
future_forecast = full_model.predict(n_periods=future_periods)

# Create future dates for the forecast
last_date = BTC1.index[-1]
future_dates = pd.date_range(start=last_date + timedelta(minutes=5), periods=future_periods)

# Create confidence intervals for future forecast
future_forecast_upper = future_forecast + 1.96 * forecast_std
future_forecast_lower = future_forecast - 1.96 * forecast_std

# Plot future forecast
plt.figure(figsize=(14, 7))
plt.plot(BTC1.index, BTC1['Close'], label='Historical Data')
plt.plot(future_dates, future_forecast, label='Forecast', color='red')
plt.fill_between(future_dates, future_forecast_lower, future_forecast_upper, color='red', alpha=0.1, label='95% Confidence Interval')
plt.title('Bitcoin Price Forecast - Future 20 Days')
plt.xlabel('Date')
plt.ylabel('Price (USD)')
plt.legend()
plt.tight_layout()
plt.show()

In [17]:
import mlflow
from mlflow.models import Model

model_uri = 'runs:/1d8707a1381d4897b8edfc2c1600f986/btc_arima_model'
# The model is logged with an input example
pyfunc_model = mlflow.pyfunc.load_model(model_uri)
input_data = pyfunc_model.input_example

# Verify the model with the provided input data using the logged dependencies.
# For more details, refer to:
# https://mlflow.org/docs/latest/models.html#validate-models-before-deployment
mlflow.models.predict(
    model_uri=model_uri,
    input_data=input_data,
    env_manager="local",
)

2025/04/19 08:06:45 INFO mlflow.models.python_api: It is highly recommended to use `uv` as the environment manager for predicting with MLflow models as its performance is significantly better than other environment managers. Run `pip install uv` to install uv. See https://docs.astral.sh/uv/getting-started/installation for other installation methods.
2025/04/19 08:06:45 INFO mlflow.models.flavor_backend_registry: Selected backend for flavor 'python_function'


{"predictions": [{"predicted_mean": 77349.3164098197}, {"predicted_mean": 77343.44206298876}, {"predicted_mean": 77337.36744910356}, {"predicted_mean": 77345.38617574351}, {"predicted_mean": 77347.96442881424}]}

c:\TS Automation\.venv\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [ ]:
import os
from sqlalchemy import create_engine

POSTGRES_HOST = os.getenv('POSTGRES_HOST')
POSTGRES_PORT = os.getenv('POSTGRES_PORT')
POSTGRES_USER = os.getenv('POSTGRES_USER')
POSTGRES_PASSWORD = os.getenv('POSTGRES_PASSWORD')
POSTGRES_DB = os.getenv('POSTGRES_DB')
POSTGRES_URL = f"postgresql://{POSTGRES_USER}:{POSTGRES_PASSWORD}@{POSTGRES_HOST}:{POSTGRES_PORT}/{POSTGRES_DB}"

# Database utilities
def save_to_db(data, table_name="btc_prices"):
    """Save data to database (placeholder function)"""
    engine = create_engine(POSTGRES_URL)
    data.to_sql(table_name, engine, if_exists='append', index=True)
    print(f"Data saved to {table_name} table")
    return True
